# Stop-RscManagedVolumeSnapshot
Managed Volumes (MV) provide hosts with a backup target location on a Rubrik cluster. The cluster manages snapshots of the data through the policies of a specified SLA Domain. DBAs will typically use a MVs for writing backups of databases where Rubrik does not have a built in snappable yet. The Stop-RscManagedVolumeSnapshot cmdlet closes the MV for write. This will then tell Rubrik to start doing its post-processing tasks like indexing.

This process works with Persistent Mount Managed Volumes and not SLA Driven Managed Volumes. 

If you are using this cmdlet, we assume that you have already written data to the MV and now need to make it read only. 

You can use any command you want to send data to the MV. 
- Oracle DBAs may use RMAN scripts
- SQL Server DBAs may use T-SQL scripts
- MySQL DBAs may use mysqldump
- PostgreSQL DBAs may use barman
- A simple copy command to copy files from a host to the share on the Rubrik cluster. 

# Workflow
To get information about the SQL Server Instance, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the Managed Volume is registered. This is akin to the datacenter that the Managed Volume is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your Managed Volume could exist on 2 different clusters. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscManagedVolume** to get the id of the Managed Volume
1. Use **Get-RscSla** to get the id of the SLA Domain. This will be used in the next command when setting the MV to read only to allow Rubrik to goveren the life cycle of the snapshot of the MV. 
1. Use **Stop-RscManagedVolumeSnapshot** to make the MV read only
1. Use **Disconnect-RSC** to disconnect from RSC

## Connect to RSC

In [1]:
Import-Module RubrikSecurityCloud
Connect-RSC


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20240219-20     v20240219-20



## Get the Cluster ID

In [2]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Managed Volume Information

In [4]:
$MVName = "rp-mysql-01"
$RscManagedVolume = Get-RscManagedVolume -Name $MVName -RscCluster $RscCluster

## Get SLA Domain Information

In [5]:
$SLADomain = "1d-14d-NRA"
$RscSlaDomain = Get-RscSla -Name $SLADomain 

## Make the Managed Volume read only

In [6]:
Stop-RscManagedVolumeSnapshot -RscManagedVolume $RscManagedVolume -RscSlaDomain $RscSlaDomain


AsyncRequestStatus                           ManagedVolumeSnapshotSummary
------------------                           ----------------------------
RubrikSecurityCloud.Types.AsyncRequestStatus 



---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [ ]:
Disconnect-Rsc